In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


## Setup

In [3]:
path = "data/icebergs/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

batch_size=64

In [4]:
batches = get_batches(path+'train', shuffle=False, batch_size=batch_size)
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

Found 1404 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [5]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 1404 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 8424 images belonging to 1 classes.


In this notebook we're going to create an ensemble of models and use their average as our predictions. For each ensemble, we're going to follow our usual fine-tuning steps:

1) Create a model that retrains just the last layer
2) Add this to a model containing all VGG layers except the last layer
3) Fine-tune just the dense layers of this model (pre-computing the convolutional layers)
4) Add data augmentation, fine-tuning the dense layers without pre-computation.

So first, we need to create our VGG model and pre-compute the output of the conv layers:

In [7]:
model = Vgg16BN().model
conv_layers,fc_layers = split_at(model, Convolution2D)

553484288/553620808 [============================>.] - ETA: 0s

In [8]:
conv_model = Sequential(conv_layers)

In [9]:
val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)
trn_features = conv_model.predict_generator(batches, batches.nb_sample)

In [10]:
save_array(model_path + 'train_convlayer_features.bc', trn_features)
save_array(model_path + 'valid_convlayer_features.bc', val_features)

In the future we can just load these precomputed features:

In [11]:
trn_features = load_array(model_path+'train_convlayer_features.bc')
val_features = load_array(model_path+'valid_convlayer_features.bc')

We can also save some time by pre-computing the training and validation arrays with the image decoding and resizing already done:

In [12]:
trn = get_data(path+'train')
val = get_data(path+'valid')

Found 1404 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [13]:
save_array(model_path+'train_data.bc', trn)
save_array(model_path+'valid_data.bc', val)

In the future we can just load these resized images:

In [14]:
trn = load_array(model_path+'train_data.bc')
val = load_array(model_path+'valid_data.bc')

Finally, we can precompute the output of all but the last dropout and dense layers, for creating the first stage of the model:

In [15]:
model.pop()
model.pop()

In [16]:
ll_val_feat = model.predict_generator(val_batches, val_batches.nb_sample)
ll_feat = model.predict_generator(batches, batches.nb_sample)

In [17]:
save_array(model_path + 'train_ll_feat.bc', ll_feat)
save_array(model_path + 'valid_ll_feat.bc', ll_val_feat)

In [18]:
ll_feat = load_array(model_path+ 'train_ll_feat.bc')
ll_val_feat = load_array(model_path + 'valid_ll_feat.bc')

...and let's also grab the test data, for when we need to submit:

In [19]:
test = get_data(path+'test')
save_array(model_path+'test_data.bc', test)

Found 8424 images belonging to 1 classes.


In [20]:
test = load_array(model_path+'test_data.bc')

## Last layer

The functions automate creating a model that trains the last layer from scratch, and then adds those new layers on to the main model.

In [21]:
def get_ll_layers():
    return [ 
        BatchNormalization(input_shape=(4096,)),
        Dropout(0.5),
        Dense(2, activation='softmax') 
        ]

In [27]:
def train_last_layer(i):
    ll_layers = get_ll_layers()
    ll_model = Sequential(ll_layers)
    ll_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    ll_model.optimizer.lr=1e-5
    ll_model.fit(ll_feat, trn_labels, validation_data=(ll_val_feat, val_labels), nb_epoch=12)
    ll_model.optimizer.lr=1e-7
    ll_model.fit(ll_feat, trn_labels, validation_data=(ll_val_feat, val_labels), nb_epoch=1)
    ll_model.save_weights(model_path+'ll_bn' + i + '.h5')

    vgg = Vgg16BN()
    model = vgg.model
    model.pop(); model.pop(); model.pop()
    for layer in model.layers: layer.trainable=False
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    ll_layers = get_ll_layers()
    for layer in ll_layers: model.add(layer)
    for l1,l2 in zip(ll_model.layers, model.layers[-3:]):
        l2.set_weights(l1.get_weights())
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model.save_weights(model_path+'bn' + i + '.h5')
    return model

## Dense model

In [28]:
def get_conv_model(model):
    layers = model.layers
    last_conv_idx = [index for index,layer in enumerate(layers) 
                         if type(layer) is Convolution2D][-1]

    conv_layers = layers[:last_conv_idx+1]
    conv_model = Sequential(conv_layers)
    fc_layers = layers[last_conv_idx+1:]
    return conv_model, fc_layers, last_conv_idx

In [29]:
def get_fc_layers(p, in_shape):
    return [
        MaxPooling2D(input_shape=in_shape),
        Flatten(),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(2, activation='softmax')
        ]

In [34]:
def train_dense_layers(i, model):
    conv_model, fc_layers, last_conv_idx = get_conv_model(model)
    conv_shape = conv_model.output_shape[1:]
    fc_model = Sequential(get_fc_layers(0.5, conv_shape))
    for l1,l2 in zip(fc_model.layers, fc_layers): 
        weights = l2.get_weights()
        l1.set_weights(weights)
    fc_model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', 
                     metrics=['accuracy'])
    fc_model.fit(trn_features, trn_labels, nb_epoch=2, 
         batch_size=batch_size, validation_data=(val_features, val_labels))

    gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.05, zoom_range=0.05, channel_shift_range=10, height_shift_range=0.05, shear_range=0.05, horizontal_flip=True)
    
        
    batches = gen.flow(trn, trn_labels, batch_size=batch_size)
    val_batches = image.ImageDataGenerator().flow(val, val_labels, 
                      shuffle=False, batch_size=batch_size)

    for layer in conv_model.layers: layer.trainable = False
    for layer in get_fc_layers(0.5, conv_shape): conv_model.add(layer)
    for l1,l2 in zip(conv_model.layers[last_conv_idx+1:], fc_model.layers): 
        l1.set_weights(l2.get_weights())

    conv_model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', 
                       metrics=['accuracy'])
    conv_model.save_weights(model_path+'no_dropout_bn' + i + '.h5')
    conv_model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=1, 
                            validation_data=val_batches, nb_val_samples=val_batches.N)
    for layer in conv_model.layers[16:]: layer.trainable = True
    conv_model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=8, 
                            validation_data=val_batches, nb_val_samples=val_batches.N)

    conv_model.optimizer.lr = 1e-7
    conv_model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=10, 
                            validation_data=val_batches, nb_val_samples=val_batches.N)
    conv_model.save_weights(model_path + 'aug' + i + '.h5')

## Build ensemble

In [35]:
for i in range(5):
    i = str(i)
    model = train_last_layer(i)
    train_dense_layers(i, model)

Train on 1404 samples, validate on 200 samples
Epoch 1/12
1404/1404 [==============================] - 0s - loss: 1.1407 - acc: 0.5242 - val_loss: 0.9654 - val_acc: 0.4200
Epoch 2/12
1404/1404 [==============================] - 0s - loss: 1.0440 - acc: 0.5726 - val_loss: 0.9836 - val_acc: 0.4100
Epoch 3/12
1404/1404 [==============================] - 0s - loss: 0.9935 - acc: 0.5698 - val_loss: 1.0313 - val_acc: 0.3850
Epoch 4/12
1404/1404 [==============================] - 0s - loss: 1.0031 - acc: 0.5798 - val_loss: 1.0929 - val_acc: 0.3500
Epoch 5/12
1404/1404 [==============================] - 0s - loss: 0.9490 - acc: 0.5805 - val_loss: 1.1611 - val_acc: 0.3350
Epoch 6/12
1404/1404 [==============================] - 0s - loss: 0.9117 - acc: 0.6075 - val_loss: 1.2192 - val_acc: 0.3250
Epoch 7/12
1404/1404 [==============================] - 0s - loss: 0.9290 - acc: 0.5919 - val_loss: 1.2941 - val_acc: 0.3200
Epoch 8/12
1404/1404 [==============================] - 0s - loss: 0.8678 - ac

## Combine ensemble and test

In [37]:
ens_model = vgg_ft_bn(2)
for layer in ens_model.layers: layer.trainable=True

In [38]:
def get_ens_pred(arr, fname):
    ens_pred = []
    for i in range(5):
        i = str(i)
        ens_model.load_weights('{}{}{}.h5'.format(model_path, fname, i))
        preds = ens_model.predict(arr, batch_size=batch_size)
        ens_pred.append(preds)
    return ens_pred

In [39]:
val_pred2 = get_ens_pred(val, 'aug')

In [40]:
val_avg_preds2 = np.stack(val_pred2).mean(axis=0)

In [41]:
categorical_accuracy(val_labels, val_avg_preds2).eval()

array(0.8100000023841858, dtype=float32)

## Generate Predictions

Let's use our new model to make predictions on the test dataset

In [54]:
test_pred2 = get_ens_pred(test, 'aug')

ValueError: too many values to unpack

In [56]:
test_avg_preds2 = np.stack(test_pred2).mean(axis=0)

In [59]:
test_avg_preds2[:5]

array([[  1.4137e-01,   8.5863e-01],
       [  9.9559e-01,   4.4107e-03],
       [  5.2689e-01,   4.7311e-01],
       [  9.2740e-01,   7.2600e-02],
       [  6.1630e-06,   9.9999e-01]], dtype=float32)

In [61]:
#Save our test results arrays so we can use them again later
save_array(model_path + 'test_preds.dat', test_avg_preds2)
save_array(model_path + 'filenames.dat', test_filenames)

## Submit Predictions to Kaggle!

Here's the format Kaggle requires for new submissions:
```
id,is_iceberg
5941774d,0.5
4023181e,0.5
b20200e4,0.5
e7f018bb,0.5
```

Kaggle wants the imageId followed by the probability of the image being an iceberg. Kaggle uses a metric called [Log Loss](http://wiki.fast.ai/index.php/Log_Loss) to evaluate your submission.

In [97]:
#Load our test predictions from file
preds=load_array(model_path + 'test_preds.dat')
filenames=load_array(model_path + 'filenames.dat')

In [98]:
isiceberg= preds[:,1]

In [90]:
isiceberg = isiceberg.clip(min=0.30, max=0.70)

In [91]:
#Extract imageIds from the filenames in our test/unknown directory 
ids = np.array([(f[8:f.find('.')]) for f in filenames])
ids[:5]

array(['0e3eb3a9', '489fccbf', '6480ab5e', 'e103cde7', 'fdf9388b'], 
      dtype='|S8')

Here we join the two columns into an array of [ids, isceberg]

In [92]:
subm = np.stack([ids,isiceberg], axis=1)
subm[:5]

array([['0e3eb3a9', '0.699999988079'],
       ['489fccbf', '0.300000011921'],
       ['6480ab5e', '0.47310590744'],
       ['e103cde7', '0.300000011921'],
       ['fdf9388b', '0.699999988079']], 
      dtype='|S32')

In [93]:
%cd ~/kaggle-iceberg/attempt2
submission_file_name = 'submission_ensemble3.csv'
np.savetxt(submission_file_name, subm, delimiter=',', header='id,is_iceberg',fmt='%s', comments='')

/home/ubuntu/kaggle-iceberg/attempt2


In [94]:
from IPython.display import FileLink
FileLink('./'+submission_file_name)

/home/ubuntu/kaggle-iceberg/attempt2/submission_ensemble3.csv

In [99]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_8 (Lambda)                (None, 3, 224, 224)   0           lambda_input_8[0][0]             
____________________________________________________________________________________________________
zeropadding2d_92 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_8[0][0]                   
____________________________________________________________________________________________________
convolution2d_92 (Convolution2D) (None, 64, 224, 224)  0           zeropadding2d_92[0][0]           
____________________________________________________________________________________________________
zeropadding2d_93 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_92[0][0]           
___________________________________________________________________________________________